In [835]:
import pybaseball
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timezone
from bs4 import BeautifulSoup
import io
import requests
import unicodedata
pybaseball.cache.enable()

In [726]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pd.read_csv(io.BytesIO(res.content), sep=',')
ID.dropna(subset=['MLBID'], inplace=True)
ID['MLBID'] = ID['MLBID'].astype(int)
BID = pd.read_csv("mlb-player-stats-Batters.csv")
PID = pd.read_csv("mlb-player-stats-P.csv")
PID['Player'] = PID['Player'].replace('Michael King', 'Mike King')

In [727]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WSH'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

In [728]:
def getDKData2024():
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata2 = pd.merge(spdata, PID[["Player", "Team"]], left_on = ["Player"], right_on = ["Player"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Player", "Team"]], left_on = ["Player"], right_on = ["Player"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    runcounter = 0

    for x in list(extract_dh["BatterTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            runcounter += 1
        else:
            new_team_list.append(x+"2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2["HomeTeam"] = ludf2["BatterTeam"].map(hometeamdict)
    ludf2["RoadTeam"] = ludf2["BatterTeam"].map(roadteamdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan
                dkdata.loc[i, 'SP'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["BatterTeam", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['BatterTeam'] = dkdata['BatterTeam'].replace('ARI', 'AZ')
    dkdata['RoadTeam'] = dkdata['RoadTeam'].replace('ARI', 'AZ')
    dkdata['HomeTeam'] = dkdata['HomeTeam'].replace('ARI', 'AZ')

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")
    dkdata = dkdata[["BatterTeam", "RoadTeam", "HomeTeam", "SP"]]

    return(dkdata)


In [729]:
#statcast(start_dt = "2022-04-07", end_dt = "2022-10-05")
#statcast(start_dt = "2023-03-30", end_dt = "2023-10-01")
savant2022 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2022.csv")
savant2023 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2023.csv")

In [730]:
#pd.set_option('display.max_columns', None)
combined1 = pd.concat([savant2022, savant2023])
combined1 = combined1[(combined1['inning'] == 1)]
combined1['game_date'] = pd.to_datetime(combined1['game_date'])
combined1['game_date'] = combined1['game_date'].dt.strftime('%Y-%m-%d')
combined1['BatterTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['away_team'], combined1['home_team'])
combined1['PitcherTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['home_team'], combined1['away_team'])
combined1["player_name"] = combined1["player_name"].apply(flip_names)
combined1['player_name'] = combined1['player_name'].replace('Michael King', 'Mike King')

In [731]:
def replace_special_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

combined1["player_name"] = combined1["player_name"].apply(replace_special_chars)

In [732]:
combined2 = combined1[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
combined2 = combined2.set_index("game_date").sort_index(ascending = True)

In [733]:
Train1 = combined2.groupby(["game_date", "BatterTeam", "away_team", "home_team", "at_bat_number", "player_name"]).agg(
    Pitches = ("at_bat_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    GB = ('bb_type', lambda x: (x == 'ground_ball').sum()),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    PU = ('bb_type', lambda x: (x == 'popup').sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    HR = ("events", lambda x: (x == 'home_run').sum()),
    Distance = ("hit_distance_sc", "mean"),
    EV = ("launch_speed", "mean"),
    LA = ("launch_angle", "mean"),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayScore = ("away_score", "sum"),
    HomeScore = ("home_score", "sum")).reset_index().fillna(0)

In [734]:
Train2 = Train1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name"]).agg(
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    GB = ('GB', "sum"),
    FB = ('FB', "sum"),
    PU = ('FB', "sum"),
    BB = ("BB", "sum"),
    HBP = ("HBP", "sum"),
    K = ("K", "sum"),
    HR = ("HR", "sum"),
    Distance = ("Distance", "mean"),
    EV = ("EV", "mean"),
    LA = ("LA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    AwayScore = ("AwayScore", "sum"),
    HomeScore = ("HomeScore", "sum")).reset_index().fillna(0)

Train2['NRFI'] = np.where((Train2['away_team'] == Train2['BatterTeam']) & (Train2['AwayScore'] == 0), 1, 0)
Train2['NRFI'] = np.where((Train2['home_team'] == Train2['BatterTeam']) & (Train2['HomeScore'] == 0), 1, Train2['NRFI'])
Train2 = Train2.drop(['AwayScore', 'HomeScore'], axis=1)

In [735]:
Train3 = Train2.groupby(["BatterTeam", "away_team", "home_team", "player_name"]).agg(
    IP = ("AB", "size"),
    AB = ("AB", "sum"),
    PA = ("PA", "sum"),
    BB = ("BB", "sum"),
    HBP = ("HBP", "sum"),
    K = ("K", "sum"),
    GB = ('GB', "sum"),
    FB = ('FB', "sum"),
    PU = ('FB', "sum"),
    HR = ("HR", "sum"),
    Distance = ("Distance", "mean"),
    EV = ("EV", "mean"),
    LA = ("LA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    NRFI = ("NRFI", "mean")).reset_index().fillna(0)

Train3['K%'] = round((Train3['K'] / Train3['AB']) * 100, 2)
Train3['BB%'] = round((Train3['BB'] / Train3['AB']) * 100, 2)
Train3["K-BB%"] = Train3['K%'] - Train3['BB%']

In [736]:
def FIP(HR, BB, HBP, K, IP):
    fip = (((HR * 13) + (3 * (BB + HBP)) - (2 * K)) / IP + 3.137)
    return round(fip, 3)

def xFIP(FB, BB, HBP, K, IP):
    lgHR = len(combined1[combined1["events"] == "home_run"])
    lgFB = len(combined1[combined1["bb_type"] == "fly_ball"])

    xfip = (13 * (FB * (lgHR/lgFB * 0.58)) + 3 * (BB + HBP) - 2 * K) / IP + 3.137
    return round(xfip, 3)

def SIERA(K, BB, GB, FB, PU, PA):
    if PA == 0:
        return 0

    so_pa = K/PA
    bb_pa = BB/PA
    gb_pa = GB/PA
    fb_pa = FB/PA
    pu_pa = PU/PA

    SIERA = 6.145 - 16.986 * (so_pa / 100) + 11.434 * (bb_pa / 100) - 1.858 * ((gb_pa - fb_pa - pu_pa) / 100) + 7.653 * (so_pa / 100) ** 2 - 6.664 * ((gb_pa - fb_pa - pu_pa) / 100) ** 2 + 10.130 * (so_pa / 100) * ((gb_pa - fb_pa - pu_pa) / 100) - 5.195 * (bb_pa / 100) * ((gb_pa - fb_pa - pu_pa) / 100)
    return round(SIERA, 3)

In [737]:
Train3['FIP'] = Train3.apply(lambda row: FIP(row['HR'], row['BB'], row['HBP'], row['K'], row['IP']), axis=1)
Train3['xFIP'] = Train3.apply(lambda row: xFIP(row['FB'], row['BB'], row['HBP'], row['K'], row['IP']), axis=1)
Train3['SIERA'] = Train3.apply(lambda row: SIERA(row['K'], row['BB'], row['GB'], row['FB'], row['PU'], row['PA']), axis=1)
Train3["AB"] = Train3["AB"] / Train3["IP"]

In [738]:
Train4 = Train3[["BatterTeam", "away_team", "home_team", "player_name", "IP", "AB", "K-BB%", "Distance", "EV", "LA", "xBA", "xwOBA", "wOBA", "RunExp", "FIP", "xFIP", "SIERA", "NRFI"]].round(3)
Train4 = Train4.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})

In [739]:
TodaysData = getDKData2024()
TodaysData

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_34346/1328962667.py:94: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_34346/1328962667.py:163: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


,BatterTeam,RoadTeam,HomeTeam,SP
Date,,,,
2024-06-08,PHI,PHI,NYM,Sean Manaea
2024-06-08,NYM,PHI,NYM,Ranger Suarez
2024-06-08,SF,SF,TEX,Andrew Heaney
2024-06-08,TEX,SF,TEX,Spencer Howard
2024-06-08,MIN,MIN,PIT,Carmen Mlodzinski
2024-06-08,PIT,MIN,PIT,Simeon Woods Richardson
2024-06-08,ATL,ATL,WSH,MacKenzie Gore
2024-06-08,WSH,ATL,WSH,Charlie Morton
2024-06-08,TOR,TOR,OAK,Luis Medina


In [740]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))
savant2024 = savant2024[(savant2024['inning'] == 1)]
savant2024['game_date'] = pd.to_datetime(savant2024['game_date'])
savant2024['game_date'] = savant2024['game_date'].dt.strftime('%Y-%m-%d')
savant2024['BatterTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['away_team'], savant2024['home_team'])
savant2024['PitcherTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['home_team'], savant2024['away_team'])
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')
savant2024["player_name"] = savant2024["player_name"].apply(flip_names)
savant2024['player_name'] = savant2024['player_name'].replace('Michael King', 'Mike King')
savant2024 = savant2024[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
savant2024 = savant2024.set_index("game_date").sort_index(ascending = True)

This is a large query, it may take a moment to complete


100%|██████████| 73/73 [01:03<00:00,  1.15it/s]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [741]:
savant2024["player_name"] = savant2024["player_name"].apply(replace_special_chars)
Season1 = savant2024[savant2024['player_name'].isin(TodaysData['SP'])]

In [742]:
Season2 = Season1.groupby(["BatterTeam", "away_team", "home_team", "at_bat_number", "player_name"]).agg(
    Pitches = ("at_bat_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    GB = ('bb_type', lambda x: (x == 'ground_ball').sum()),
    FB = ('bb_type', lambda x: (x == 'fly_ball').sum()),
    PU = ('bb_type', lambda x: (x == 'popup').sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    HR = ("events", lambda x: (x == 'home_run').sum()),
    Distance = ("hit_distance_sc", "mean"),
    EV = ("launch_speed", "mean"),
    LA = ("launch_angle", "mean"),
    xBA = ("estimated_ba_using_speedangle", "mean"),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean"),
    AwayScore = ("away_score", "sum"),
    HomeScore = ("home_score", "sum")).reset_index().fillna(0)

In [743]:
Season3 = Season2.groupby(["BatterTeam", "away_team", "home_team", "player_name"]).agg(
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    GB = ('GB', "sum"),
    FB = ('FB', "sum"),
    PU = ('FB', "sum"),
    BB = ("BB", "sum"),
    HBP = ("HBP", "sum"),
    K = ("K", "sum"),
    HR = ("HR", "sum"),
    Distance = ("Distance", "mean"),
    EV = ("EV", "mean"),
    LA = ("LA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    AwayScore = ("AwayScore", "sum"),
    HomeScore = ("HomeScore", "sum")).reset_index().fillna(0)

Season3['NRFI'] = np.where((Season3 ['away_team'] == Season3 ['BatterTeam']) & (Season3['AwayScore'] == 0), 1, 0)
Season3['NRFI'] = np.where((Season3['home_team'] == Season3['BatterTeam']) & (Season3['HomeScore'] == 0), 1, Season3['NRFI'])
Season3 = Season3.drop(['AwayScore', 'HomeScore'], axis=1)

In [744]:
Season4 = Season3.groupby(["player_name"]).agg(
    IP = ("AB", "size"),
    AB = ("AB", "sum"),
    PA = ("PA", "sum"),
    BB = ("BB", "sum"),
    HBP = ("HBP", "sum"),
    K = ("K", "sum"),
    GB = ('GB', "sum"),
    FB = ('FB', "sum"),
    PU = ('FB', "sum"),
    HR = ("HR", "sum"),
    Distance = ("Distance", "mean"),
    EV = ("EV", "mean"),
    LA = ("LA", "mean"),
    xBA = ("xBA", "mean"),
    xwOBA = ("xwOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean"),
    NRFI = ("NRFI", "mean")).reset_index().fillna(0)

Season4['K%'] = round((Season4['K'] / Season4['AB']) * 100, 2)
Season4['BB%'] = round((Season4['BB'] / Season4['AB']) * 100, 2)
Season4["K-BB%"] = Season4['K%'] - Season4['BB%']

In [745]:
Season4['FIP'] = Season4.apply(lambda row: FIP(row['HR'], row['BB'], row['HBP'], row['K'], row['IP']), axis=1)
Season4['xFIP'] = Season4.apply(lambda row: xFIP(row['FB'], row['BB'], row['HBP'], row['K'], row['IP']), axis=1)
Season4['SIERA'] = Season4.apply(lambda row: SIERA(row['K'], row['BB'], row['GB'], row['FB'], row['PU'], row['PA']), axis=1)
Season4["AB"] = Season4["AB"] / Season4["IP"]
Season5 = Season4[["player_name", "IP", "AB", "K-BB%", "Distance", "EV", "LA", "xBA", "xwOBA", "wOBA", "RunExp", "FIP", "xFIP", "SIERA", "NRFI"]].round(3)

In [746]:
TodaysData.dropna(subset=['SP'], inplace=True)
TodaysData1 = pd.merge(TodaysData, Season5[["player_name", "IP", "AB", "K-BB%", "Distance", "EV", "LA", "xBA", "xwOBA", "wOBA", "RunExp", "FIP", "xFIP", "SIERA", "NRFI"]], left_on = ['SP'], right_on = ['player_name'], how = 'left').drop(columns = ["player_name"])
TodaysData1['IP'] = TodaysData1['IP'].astype(float)

# If no 2024 savant data exists then gives them the league averages from 2022-23
TrainMeans = Train4.drop(['BatterTeam', 'RoadTeam', "HomeTeam", "SP"], axis=1).mean()
TodaysData1 = TodaysData1.fillna(TrainMeans)
TodaysData1

,BatterTeam,RoadTeam,HomeTeam,SP,IP,AB,K-BB%,Distance,EV,LA,xBA,xwOBA,wOBA,RunExp,FIP,xFIP,SIERA,NRFI
0,PHI,PHI,NYM,Sean Manaea,11.000000,3.636,27.5,168.423,76.877,20.258,0.21,0.263,0.152,-0.04,1.228000,2.372000,6.100000,0.9090
1,NYM,PHI,NYM,Ranger Suarez,12.000000,4.0,22.92,127.427,69.79,15.522,0.17,0.246,0.265,0.037,4.054000,2.470000,6.093000,0.7500
2,SF,SF,TEX,Andrew Heaney,11.000000,4.364,12.5,154.024,73.634,19.368,0.179,0.266,0.233,-0.023,3.773000,3.990000,6.117000,0.7270
3,TEX,SF,TEX,Spencer Howard,1.288639,3.975117,16.981499,137.681962,72.214554,14.860904,0.207956,0.263504,0.276171,-0.006929,4.324255,3.535813,6.103584,0.7158
4,MIN,MIN,PIT,Carmen Mlodzinski,1.288639,3.975117,16.981499,137.681962,72.214554,14.860904,0.207956,0.263504,0.276171,-0.006929,4.324255,3.535813,6.103584,0.7158
5,PIT,MIN,PIT,Simeon Woods Richardson,9.000000,3.333,33.34,131.259,72.483,17.315,0.116,0.152,0.064,-0.05,1.359000,1.981000,6.079000,1.0000
6,ATL,ATL,WSH,MacKenzie Gore,12.000000,3.917,34.04,149.122,66.479,18.584,0.13,0.196,0.206,-0.018,1.887000,1.620000,6.077000,0.9170
7,WSH,ATL,WSH,Charlie Morton,11.000000,4.727,21.15,99.028,65.225,-2.418,0.173,0.268,0.271,0.008,3.592000,2.791000,6.090000,0.8180
8,TOR,TOR,OAK,Luis Medina,1.000000,5.0,0.0,128.5,71.67,11.5,0.146,0.292,0.32,0.021,4.137000,4.137000,6.131000,0.0000
9,OAK,TOR,OAK,Kevin Gausman,12.000000,4.25,7.85,153.215,73.958,17.084,0.261,0.396,0.309,0.005,6.220000,4.136000,6.126000,0.7500


In [998]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train4.copy()
TodaysData2 = TodaysData1.copy()

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

In [999]:
TrainFeatures = Train5.drop(columns = ["IP", "NRFI"]).values.reshape(-1, 16)
TrainLabel = Train5["NRFI"].values.reshape(-1, 1)
TodayFeatures = TodaysData2.drop(columns = ["IP", "NRFI"]).values.reshape(-1, 16)

rf_regressor = RandomForestRegressor(n_estimators=150, max_leaf_nodes = 30, max_depth = 5)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["RFPred"] = RFpred

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [1000]:
gb_regressor = GradientBoostingRegressor(n_estimators=175, min_samples_leaf = 2, max_depth=3, max_features='log2', learning_rate=0.05)
gb_regressor.fit(TrainFeatures, TrainLabel)
GBpred = gb_regressor.predict(TodayFeatures)

TodaysData2["GBPred"] = GBpred

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [1001]:
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        # Handling default value of 536
        TodaysData2[col] = TodaysData2[col].apply(lambda x: label_encoder.inverse_transform([x])[0] if x != 536 else np.nan)

TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_34346/1190844444.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [1002]:
TodaysData2

,BatterTeam,RoadTeam,HomeTeam,SP,IP,AB,K-BB%,Distance,EV,LA,xBA,xwOBA,wOBA,RunExp,FIP,xFIP,SIERA,NRFI,RFPred,GBPred
0,PHI,PHI,NYM,Sean Manaea,11.000000,3.636,27.5,168.423,76.877,20.258,0.21,0.263,0.152,-0.04,1.228000,2.372000,6.100000,0.9090,0.974885,0.965942
1,NYM,PHI,NYM,Ranger Suarez,12.000000,4.0,22.92,127.427,69.79,15.522,0.17,0.246,0.265,0.037,4.054000,2.470000,6.093000,0.7500,0.850979,0.764094
2,SF,SF,TEX,Andrew Heaney,11.000000,4.364,12.5,154.024,73.634,19.368,0.179,0.266,0.233,-0.023,3.773000,3.990000,6.117000,0.7270,0.828600,0.814822
3,TEX,SF,TEX,Spencer Howard,1.288639,3.975117,16.981499,137.681962,72.214554,14.860904,0.207956,0.263504,0.276171,-0.006929,4.324255,3.535813,6.103584,0.7158,0.893737,0.836865
4,MIN,MIN,PIT,Carmen Mlodzinski,1.288639,3.975117,16.981499,137.681962,72.214554,14.860904,0.207956,0.263504,0.276171,-0.006929,4.324255,3.535813,6.103584,0.7158,0.893737,0.834385
5,PIT,MIN,PIT,Simeon Woods Richardson,9.000000,3.333,33.34,131.259,72.483,17.315,0.116,0.152,0.064,-0.05,1.359000,1.981000,6.079000,1.0000,0.974885,0.978922
6,ATL,ATL,WSH,MacKenzie Gore,12.000000,3.917,34.04,149.122,66.479,18.584,0.13,0.196,0.206,-0.018,1.887000,1.620000,6.077000,0.9170,0.974126,0.950592
7,WSH,ATL,WSH,Charlie Morton,11.000000,4.727,21.15,99.028,65.225,-2.418,0.173,0.268,0.271,0.008,3.592000,2.791000,6.090000,0.8180,0.599648,0.631573
8,TOR,TOR,OAK,Luis Medina,1.000000,5.0,0.0,128.5,71.67,11.5,0.146,0.292,0.32,0.021,4.137000,4.137000,6.131000,0.0000,0.541928,0.596617
9,OAK,TOR,OAK,Kevin Gausman,12.000000,4.25,7.85,153.215,73.958,17.084,0.261,0.396,0.309,0.005,6.220000,4.136000,6.126000,0.7500,0.584219,0.687770


In [1018]:
TodaysData3 = TodaysData2.groupby(["RoadTeam", "HomeTeam"])[['RFPred', "GBPred"]].prod().reset_index()

TodaysData3['Games'] = TodaysData3.apply(lambda x: tuple(sorted([x['RoadTeam'], x['HomeTeam']])), axis=1)
TodaysData3['Score'] = TodaysData3[['RFPred', 'GBPred']].mean(axis=1)
TodaysData4 = TodaysData3.set_index("Games").drop(columns = ["RoadTeam", "HomeTeam"]).sort_values("Score", ascending = False).drop(columns = ["RFPred", "GBPred"]).round(3)
TodaysData4

,Score
Games,
"(KC, SEA)",0.916
"(MIN, PIT)",0.844
"(NYM, PHI)",0.784
"(LAD, NYY)",0.781
"(SF, TEX)",0.711
"(DET, MIL)",0.680
"(BAL, TB)",0.652
"(BOS, CWS)",0.616
"(ATL, WSH)",0.592


In [637]:
excel_file = 'todays_data.xlsx'

# If deleting the mode and engine it rewrites the whole file
with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl') as writer:
    TodaysData4.reset_index().to_excel(writer, index=False, sheet_name='6-9-24')